# Assignment Chapter 3 - COMPUTER VISION [Case #3]
Startup Campus, Indonesia - `Artificial Intelligence (AI)` (Batch 7)
* Dataset: Any YouTube videos
* Libraries: PyTorch, Numpy, OpenCV2
* Objective: Real-time Object Detection using CNN-based Pre-trained Models

`PERSYARATAN` Semua modul (termasuk versi yang sesuai) sudah di-install dengan benar.
<br>`CARA PENGERJAAN` Lengkapi baris kode yang ditandai dengan **#TODO**.
<br>`TARGET PORTFOLIO` Peserta mampu mengimplementasikan model PyTorch untuk mendeteksi objek secara *real-time*.

### Install additional library

In [ ]:
!pip install cap-from-youtube

### Import Libraries

In [ ]:
import torch
import numpy as np
import cv2

from cap_from_youtube import cap_from_youtube
from time import time
from datetime import datetime as dt

### User-defined Class

In [ ]:
class ObjectDetection:
    def __init__(self, url, out_file="{}_video.avi".format(dt.now().strftime("%Y%m%d_%H%M%S"))):
        """
        Initializes the class with youtube url and output file.
        :param url: Has to be as youtube URL,on which prediction is made.
        :param out_file: A valid output file name.
        """

        self._URL = url
        self.model = self.load_model()
        self.classes = self.model.names
        self.out_file = out_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def get_video_from_url(self):
        """
        Creates a new video streaming object to extract video frame by frame to make prediction on.
        :return: opencv2 video capture object, with lowest quality frame available for video.
        """

        return cap_from_youtube(self._URL)

    def load_model(self):
        """
        Loads the model from pytorch hub.
        :return: Trained Pytorch model.
        """

        # TODO: Panggil model ultralytics/yolov5
        # Lihat caranya di https://pytorch.org/hub/ultralytics_yolov5/#load-from-pytorch-hub
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame using yolo5 model.
        :param frame: input frame in numpy/list/tuple format.
        :return: Labels and Coordinates of objects detected by model in the frame.
        """

        self.model.to(self.device)

        frame = [frame]
        results = self.model(frame)
        labels, cord = results.xyxyn[0][:, -1].cpu().numpy(), results.xyxyn[0][:, :-1].cpu().numpy()
        return labels, cord

    def class_to_label(self, x):
        """
        For a given label value, return corresponding string label.
        :param x: numeric label
        :return: corresponding string label
        """

        return self.classes[int(x)]

    def plot_boxes(self, results, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :return: Frame with bounding boxes and labels ploted on it.
        """

        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, self.class_to_label(labels[i]), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)

        return frame

    def __call__(self):
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :return: void
        """

        player = self.get_video_from_url()
        assert player.isOpened()

        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.out_file, four_cc, 20, (x_shape, y_shape))

        for i in range(1, 300):
            start_time = time()
            ret, frame = player.read()

            results = self.score_frame(frame)
            frame = self.plot_boxes(results, frame)
            end_time = time()

            fps = 1/np.round(end_time - start_time, 3)
            print(f"Frames Per Second : {fps}")
            out.write(frame)

### IMPORTANT: Activate your GPU

- Di Google Colab, klik **Runtime > Change runtime time**, lalu silakan pilih **T4 GPU**.

### Start Object Detection

In [ ]:
if __name__ == "__main__":
    # Pastikan CUDA enable: TRUE
    print("CUDA enable: {}".format(torch.cuda.is_available()))

    # TODO: Isi parameter dengan URL YouTube yang tersedia (secara bergantian):
    # 1. Crowded place: https://www.youtube.com/watch?v=dwD1n7N7EAg
    # 2. Solar system: https://www.youtube.com/watch?v=g2KmtA97HxY
    # 3. Road traffic: https://www.youtube.com/watch?v=wqctLW0Hb_0

    run_model = ObjectDetection(url="https://www.youtube.com/watch?v=dwD1n7N7EAg")
    run_model()

CUDA enable: True


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-26 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


[youtube] Extracting URL: https://www.youtube.com/watch?v=dwD1n7N7EAg
[youtube] dwD1n7N7EAg: Downloading webpage
[youtube] dwD1n7N7EAg: Downloading ios player API JSON
[youtube] dwD1n7N7EAg: Downloading mweb player API JSON
[youtube] dwD1n7N7EAg: Downloading m3u8 information


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
WARNING ⚠️ NMS time limit 0.550s exceeded
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Frames Per Second : 1.0493179433368311
Frames Per Second : 43.47826086956522
Frames Per Second : 41.666666666666664
Frames Per Second : 43.47826086956522
Frames Per Second : 41.666666666666664


...

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.am

Frames Per Second : 45.45454545454546
Frames Per Second : 52.631578947368425
Frames Per Second : 50.0
Frames Per Second : 43.47826086956522
Frames Per Second : 47.61904761904761


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Frames Per Second : 43.47826086956522
Frames Per Second : 37.03703703703704
Frames Per Second : 52.631578947368425
Frames Per Second : 40.0
Frames Per Second : 43.47826086956522
Frames Per Second : 50.0


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [ ]:
# [ PERTANYAAN ]
# TODO: Apa perbedaan "image classification" dan "object detection"?

[ ANSWER HERE ]

Image classification dan object detection merupakan dua teknik yang memiliki tujuan berbeda dalam computer vision.
**Image classification** fokus pada pengidentifikasian isi utama dari sebuah gambar secara keseluruhan. Misalnya, ketika kita memperlihatkan sebuah foto kepada komputer, image classification akan memberikan satu jawaban kategori seperti "ini adalah dapur", "ini adalah foto kucing", "ini anjing" dan lain lain. Sedangkan **Object detection** lebih kompleks, teknik ini tidak hanya mengenali objek dalam gambar, tetapi juga dapat menentukan lokasi dan jumlah objek tersebut. Misalkan seperti foto dapur, object detection dapat mengenali dan menandai  posisi "ada kompor di sebelah kiri, kulkas di pojok kanan, dan rak piring di tengah ruangan.", setiap objek akan ditandai dengan kotak pembatas yang menunjukkan lokasinya secara spesifik.

Perbedaan utamanya terletak pada detail informasi yang dihasilkan, dimana image classification hanya memberikan label untuk keseluruhan gambar, sedangkan object detection memberikan label serta posisi setiap objek yang terdeteksi dalam gambar.

In [ ]:
# [ PERTANYAAN ]
# TODO: Di video mana YOLOv5 memiliki akurasi deteksi terburuk? Mengapa?

[ ANSWER HERE ]

YOLOv5 menunjukkan akurasi deteksi yang terburuk pada video ***crowded place***. Karena jumlah objek yang sangat banyak dan saling berdekatan dalam satu frame, yang membuat model kesulitan untuk membedakan setiap objek dengan jelas. Sehingga, YOLOv5 sering mengalami *overlapping* deteksi, di mana beberapa objek terlihat tumpang tindih atau bahkan tidak terdeteksi sama sekali.

Selain itu, terdapat penurunan kecepatan pemrosesan (*Frame Per Second atau FPS*) yang fluktuatif, kadang mencapai angka yang sangat rendah, sehingga deteksi berjalan lebih lambat. Ini membuat YOLOv5 memerlukan waktu lebih lama, yang akhirnya memengaruhi akurasinya secara keseluruhan. Dibandingkan dengan video lain, seperti video *Solar system*  atau *Road traffic*, YOLOv5 mampu bekerja lebih baik karena objek-objek dalam video tersebut lebih mudah dikenali (jelas) dan terpisah.

### Scoring
Total `#TODO` = 4
<br>Checklist:

- [ ✔] Panggil model ultralytics/yolov5
- [ ✔] Isi parameter dengan URL YouTube yang tersedia
- [ ✔] [ PERTANYAAN ] Apa perbedaan "image classification" dan "object detection"?
- [ ✔] [ PERTANYAAN ] Di video mana YOLOv5 memiliki akurasi deteksi terburuk? Mengapa?

### Additional readings
* N/A

### Copyright © 2024 Startup Campus, Indonesia
* Prepared by **Nicholas Dominic, M.Kom.** [(profile)](https://linkedin.com/in/nicholas-dominic)
* You may **NOT** use this file except there is written permission from PT. Kampus Merdeka Belajar (Startup Campus).
* Please address your questions to mentors.